# Creando clientes

Para crear un cliente podemos enviar el dni, nombre y apellido a una petición a la ruta `/clientes/crear/`, pero esta vez en lugar del método `GET` utilizaremos `POST` para enviar datos de formulario que no son visibles en la ruta de la petición.

A la hora de enviar varios campos relacionados para un único objeto, `FastAPI` hace uso de una biblioteca muy extendida llamada `pydantic`, ésta permite crear modelos de datos y añadirles validaciones. 

Crearemos un `ModeloCliente` para la clase `Cliente`, también importaremos `constr` para definir las validaciones obligatorias. Podéis consultar más información sobre ello [en la documentación](https://pydantic-docs.helpmanual.io/usage/types/#constrained-types) de `pydantic`:

```python

from pydantic import BaseModel, constr

class ModeloCliente(BaseModel):
    dni: constr(min_length=3, max_length=3)
    nombre: constr(min_length=2, max_length=30)
    apellido: constr(min_length=2, max_length=30)
```

Para validar el DNI con nuestra función especial podemos crear una subclase `ModeloCliente` heredando del `ModeloCrearCliente`, luego veréis porqué. En ella añadiremos un método `validar_dni` utilizando el decorador `validator`:

```python
class ModeloCrearCliente(ModeloCliente):
    @validator("dni")
    def validar_dni(cls, dni):
        if not helpers.dni_valido(dni, db.Clientes.lista):
            raise ValueError("Cliente ya existente o DNI incorrecto")
        return dni
```

Utilizaremos el modelo como plantilla para recuperar los datos de la petición `POST` y crear el cliente, dando por hecho que todas las validaciones son correctas antes de llegar a este punto:

```python
@app.post("/clientes/crear/")
async def clientes_crear(datos: ModeloCrearCliente):
    cliente = db.Clientes.crear(datos.dni, datos.nombre, datos.apellido)
    if cliente:
        headers = {"content-type": "charset=utf-8"}
        return JSONResponse(content=content.to_dict(), headers=headers)
    raise HTTPException(status_code=404)
```

Para probar la petición utilizaremos la plataforma de documentación, cambiando el contenido acordemente. 

Si alguna validación del modelo creado con `pydantic` no es correcta:

```json
{
  "dni": "",
  "nombre": "",
  "apellido": ""
}
```

Se devolverá automáticamente una respuesta autogenerada:

```json
{
  "detail": [
    {
      "loc": [
        "body",
        "dni"
      ],
      "msg": "ensure this value has at least 3 characters",
      "type": "value_error.any_str.min_length",
      "ctx": {
        "limit_value": 3
      }
    },
    {
      "loc": [
        "body",
        "nombre"
      ],
      "msg": "ensure this value has at least 2 characters",
      "type": "value_error.any_str.min_length",
      "ctx": {
        "limit_value": 2
      }
    },
    {
      "loc": [
        "body",
        "apellido"
      ],
      "msg": "ensure this value has at least 2 characters",
      "type": "value_error.any_str.min_length",
      "ctx": {
        "limit_value": 2
      }
    }
  ]
}
```

Si las validaciones del modelo están bien pero falla la validación para el DNI se devolverá la respuesta que hemos configurado:

```json
{
  "detail": [
    {
      "loc": [
        "body",
        "dni"
      ],
      "msg": "Cliente ya existente o DNI incorrecto",
      "type": "value_error"
    }
  ]
}
```

Si todo está correcto se creará el cliente y se devolverá su información:

```json
{"dni":"36N","nombre":"Fernando","apellido":"López"}
```

Además se añadirá al fichero CSV:

```csv
15J;Marta;Pérez
48H;Manolo;López
28Z;Mariana;García
36N;Fernando;López
```

Con esto finalizamos la función de creación.